In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
import numpy as np

C:\Users\Ignatius Ting\AppData\Local\Temp\ipykernel_9600\3472114568.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [2]:
zz = pd.read_json('../data/modelling_enc/rel_val_v1.json', lines = True)

In [4]:
zz.columns

Index(['feature_rel', 'target_rel'], dtype='object')

In [6]:
len(zz.target_rel.unique())

120

In [7]:
aa = pd.read_json('../data/modelling_enc/rel_train_v1.json', lines = True)
len(aa.target_rel.unique())

120

In [8]:
aa.shape

(30543, 2)

In [92]:
train = pd.read_csv('../data/modelling/v2_data/df_train_v2.csv')
test = pd.read_csv('../data/modelling/v2_data/df_test_v2.csv')
val = pd.read_csv('../data/modelling/v2_data/df_val_v2.csv')

In [93]:
df = pd.concat([train,test,val])
df.reset_index(inplace = True, drop = True)


df['target'] = df.apply(lambda x: [x['rel_1_clean'], x['rel_2_clean']] if pd.isna(x['rel_3_clean'])\
                              else [x['rel_1_clean'], x['rel_2_clean'], x['rel_3_clean']], axis = 1)

In [94]:
# get adequate class distribution
temp = df.target.value_counts()
targets = temp[temp > 4].index.tolist()
print(df.shape)
df = df[df.target.isin(targets)].reset_index(drop = True)
print(zz.shape)

(25116, 27)
(22777, 29)


In [95]:
def create_triples(row):
    triples = []
    
    for i in range(1,4):
        if i == 1:
            source_col = row[f'source_text']
        else:
            source_col = row[f'cui_{i}_text']
        cui_col = row[f'cui_{i+1}_text']
        rel_col = row[f'rel_{i}_clean']
        
        if not pd.isna(rel_col):
            triples.append((source_col, rel_col, cui_col))
    
    return triples

df['triples'] = df.apply(create_triples, axis=1)
df['triples'] = df['triples'].apply(lambda x: [triple for triple in x if triple])  # Remove empty lists
df['triples'] = df['triples'].apply(lambda x: tuple(x))
df_t5 = df.copy()
# print(df[['source_text', 'triples']])

In [96]:
df['explode_key'] = df['triples'].copy()
df_rel = df.explode('explode_key')
for x in df_rel.explode_key:
    assert len(x) == 3

In [97]:
df_rel['target_rel'] = df_rel.explode_key.apply(lambda x: x[1])
df_rel['entities'] = df_rel.explode_key.apply(lambda x: [x[0],x[2]])
df_rel['feature_rel'] = df_rel.apply(lambda x: f"{x['entities']} [SEP] {x['result']}", axis = 1)


temp = df_rel.target_rel.value_counts()
targets = temp[temp > 4].index.tolist()
print(df_rel.shape)
# get triples of relations that fall under threshold
triples_rm = df_rel[~df_rel.target_rel.isin(targets)].triples.unique()
# # Convert the inner lists to tuples
# unique_tuples = {tuple(inner_list) for sublist in triples_rm for inner_list in sublist}

# # Convert the unique tuples back to lists
# triples_rm = [list(inner_tuple) for inner_tuple in unique_tuples]
df_rel = df_rel[~df_rel.triples.isin(triples_rm)]
print(df_rel.shape)


(50901, 32)
(50901, 32)


In [98]:
# we don't want to spread instances of the same triple across train/val/test
# take one instance of each triple set, and then do split
# then we can rejoin the individual rel targets after
df_rel_split = df_rel.drop_duplicates('triples', keep = 'first')

In [99]:
from sklearn.model_selection import train_test_split
def custom_train_test_validation_split(df, train_ratio=0.6, test_ratio=0.2, validation_ratio=0.2, random_state=None):
    # First, split the data into training and temporary (test + validation) sets
    df_train, df_temp = train_test_split(df, test_size=(test_ratio + validation_ratio), random_state=random_state, stratify=df['target'])
    
    # Then, split the temporary set into test and validation sets
    relative_validation_ratio = validation_ratio / (test_ratio + validation_ratio)
    df_test, df_val = train_test_split(df_temp, test_size=relative_validation_ratio, random_state=random_state, stratify=df_temp['target'])
    
    return df_train, df_test, df_val

In [107]:
rel_train, rel_test, rel_val = custom_train_test_validation_split(df = df_rel_split)

In [108]:
rel_train.shape

(13665, 32)

In [103]:
df_rel.columns

Index(['source_cui', 'relation 1', 'cui_2', 'relation 2', 'cui_3', 'query',
       'url', 'result', 'source_text', 'cui_2_text', 'cui_3_text',
       'source_clean', 'cui_2_clean', 'cui_3_clean', 'rel_1_clean',
       'rel_2_clean', 'rel_combine', 'source', 'target', 'strat_split',
       'relation', 'relation_2', 'relation_3', 'cui_4', 'cui_4_text',
       'cui_4_clean', 'rel_3_clean', 'triples', 'explode_key', 'target_rel',
       'entities', 'feature_rel'],
      dtype='object')

In [109]:
rel_train = df_rel.merge(rel_train[['triples']], on = 'triples', how = 'inner')
rel_test = df_rel.merge(rel_test[['triples']], on = 'triples', how = 'inner')
rel_val = df_rel.merge(rel_val[['triples']], on = 'triples', how = 'inner')

In [114]:
rel_train.columns

Index(['source_cui', 'relation 1', 'cui_2', 'relation 2', 'cui_3', 'query',
       'url', 'result', 'source_text', 'cui_2_text', 'cui_3_text',
       'source_clean', 'cui_2_clean', 'cui_3_clean', 'rel_1_clean',
       'rel_2_clean', 'rel_combine', 'source', 'target', 'strat_split',
       'relation', 'relation_2', 'relation_3', 'cui_4', 'cui_4_text',
       'cui_4_clean', 'rel_3_clean', 'triples', 'explode_key', 'target_rel',
       'entities', 'feature_rel'],
      dtype='object')

In [115]:
import json
import os

def write_df_to_json(dataframe, filename, data_dir='../data/', encoding='utf-8'):
    file_path = os.path.join(data_dir, filename)

    with open(file_path, 'w', encoding=encoding) as json_file:
        # Iterate over the rows of the DataFrame
        for _, row in dataframe.iterrows():
            # Convert the row to a dictionary and write it to the file
            json.dump(row.to_dict(), json_file, ensure_ascii=False)
            # Add a newline character to separate the JSON objects
            json_file.write('\n')
# Example usage:
write_df_to_json(rel_train[['feature_rel','target_rel']], 'rel_train_v1.json', data_dir = '../data/modelling_enc/')
write_df_to_json(rel_val[['feature_rel','target_rel']], 'rel_val_v1.json', data_dir = '../data/modelling_enc/')
write_df_to_json(rel_test[['feature_rel','target_rel']], 'rel_test_v1.json', data_dir = '../data/modelling_enc/')

write_df_to_json(rel_train, 'full_rel_train_v1.json', data_dir = '../data/modelling_enc/')
write_df_to_json(rel_val, 'full_val_v1.json', data_dir = '../data/modelling_enc/')
write_df_to_json(rel_test, 'full_test_v1.json', data_dir = '../data/modelling_enc/')

### Copy for encoder-decoder

In [7]:
df_train_t5 = df_train.copy()
df_test_t5 = df_test.copy()
df_val_t5 = df_val.copy()

In [8]:
df_train['feature'] = df_train['source'].apply(lambda x: x.replace("</s>", "[SEP]"))
df_test['feature'] = df_test['source'].apply(lambda x: x.replace("</s>", "[SEP]"))
df_val['feature'] = df_val['source'].apply(lambda x: x.replace("</s>", "[SEP]"))

### Write encoder data to JSON

In [9]:
import json
import os

train_json = df_train[['feature','target']]
val_json = df_val[['feature','target']]
test_json = df_test[['feature','target']]

data_dir = '../data/modelling_enc/'

train_file = os.path.join(data_dir,'train_v3.json')
val_file = os.path.join(data_dir,'val_v3.json')
test_file = os.path.join(data_dir,'test_v3.json')


with open(train_file, 'w', encoding = 'utf-8') as train_json_file:
    # Iterate over the rows of the DataFrame
    for index, row in train_json.iterrows():
        # Convert the row to a dictionary and write it to the file
        json.dump(row.to_dict(), train_json_file, ensure_ascii = False)
        # Add a newline character to separate the JSON objects
        train_json_file.write('\n')

with open(val_file, 'w', encoding = 'utf-8') as val_json_file:
    # Iterate over the rows of the DataFrame
    for index, row in val_json.iterrows():
        # Convert the row to a dictionary and write it to the file
        json.dump(row.to_dict(), val_json_file, ensure_ascii = False)
        # Add a newline character to separate the JSON objects
        val_json_file.write('\n')

with open(test_file, 'w', encoding = 'utf-8') as test_json_file:
    # Iterate over the rows of the DataFrame
    for index, row in test_json.iterrows():
        # Convert the row to a dictionary and write it to the file
        json.dump(row.to_dict(), test_json_file, ensure_ascii = False)
        # Add a newline character to separate the JSON objects
        test_json_file.write('\n')





### Encoder-Decoder

In [10]:
df_train_t5.rename(columns = {'source':'feature'}, inplace = True)
df_test_t5.rename(columns = {'source':'feature'}, inplace = True)
df_val_t5.rename(columns = {'source':'feature'}, inplace = True)

In [11]:
df_train_t5['target'] = df_train_t5.apply(lambda x: f"[['{x['source_clean']}', '{x['rel_1_clean']}', '{x['cui_2_clean']}'], ['{x['cui_2_clean']}', '{x['rel_2_clean']}', '{x['cui_3_clean']}']]" \
                        if pd.isna(x['rel_3_clean']) else\
                        f"[['{x['source_clean']}', '{x['rel_1_clean']}', '{x['cui_2_clean']}'], ['{x['cui_2_clean']}', '{x['rel_2_clean']}', '{x['cui_3_clean']}'], ['{x['cui_3_clean']}', '{x['rel_3_clean']}', '{x['cui_4_clean']}']]", 
                        axis = 1)
df_test_t5['target'] = df_test_t5.apply(lambda x: f"[['{x['source_clean']}', '{x['rel_1_clean']}', '{x['cui_2_clean']}'], ['{x['cui_2_clean']}', '{x['rel_2_clean']}', '{x['cui_3_clean']}']]" \
                        if pd.isna(x['rel_3_clean']) else\
                        f"[['{x['source_clean']}', '{x['rel_1_clean']}', '{x['cui_2_clean']}'], ['{x['cui_2_clean']}', '{x['rel_2_clean']}', '{x['cui_3_clean']}'], ['{x['cui_3_clean']}', '{x['rel_3_clean']}', '{x['cui_4_clean']}']]", 
                        axis = 1)
df_val_t5['target'] = df_val_t5.apply(lambda x: f"[['{x['source_clean']}', '{x['rel_1_clean']}', '{x['cui_2_clean']}'], ['{x['cui_2_clean']}', '{x['rel_2_clean']}', '{x['cui_3_clean']}']]" \
                        if pd.isna(x['rel_3_clean']) else\
                        f"[['{x['source_clean']}', '{x['rel_1_clean']}', '{x['cui_2_clean']}'], ['{x['cui_2_clean']}', '{x['rel_2_clean']}', '{x['cui_3_clean']}'], ['{x['cui_3_clean']}', '{x['rel_3_clean']}', '{x['cui_4_clean']}']]", 
                        axis = 1)

In [16]:
df_train[['target']].sample(1)

,target
11105,"[has finding site, finding site of]"


In [17]:
df_train_t5.to_csv('../data/modelling/train_v3.csv', index = False)
df_test_t5.to_csv('../data/modelling/test_v3.csv', index = False)
df_val_t5.to_csv('../data/modelling/val_v3.csv', index = False)

In [18]:
df_train_t5.shape

(13666, 27)

In [21]:
df_train_t5.feature.isna().mean()

0.0

In [19]:
df_train.shape

(13666, 28)

In [22]:
df_train.feature.isna().mean()

0.0

In [12]:
import json
import os

train_json = df_train_t5[['feature','target']]
val_json = df_test_t5[['feature','target']]
test_json = df_val_t5[['feature','target']]

data_dir = '../data/modelling/'

train_file = os.path.join(data_dir,'train_v3.json')
val_file = os.path.join(data_dir,'val_v3.json')
test_file = os.path.join(data_dir,'test_v3.json')


with open(train_file, 'w', encoding = 'utf-8') as train_json_file:
    # Iterate over the rows of the DataFrame
    for index, row in train_json.iterrows():
        # Convert the row to a dictionary and write it to the file
        json.dump(row.to_dict(), train_json_file, ensure_ascii = False)
        # Add a newline character to separate the JSON objects
        train_json_file.write('\n')

with open(val_file, 'w', encoding = 'utf-8') as val_json_file:
    # Iterate over the rows of the DataFrame
    for index, row in val_json.iterrows():
        # Convert the row to a dictionary and write it to the file
        json.dump(row.to_dict(), val_json_file, ensure_ascii = False)
        # Add a newline character to separate the JSON objects
        val_json_file.write('\n')

with open(test_file, 'w', encoding = 'utf-8') as test_json_file:
    # Iterate over the rows of the DataFrame
    for index, row in test_json.iterrows():
        # Convert the row to a dictionary and write it to the file
        json.dump(row.to_dict(), test_json_file, ensure_ascii = False)
        # Add a newline character to separate the JSON objects
        test_json_file.write('\n')



